# Coroutine

In [1]:
def A():
    print('1')
    print('2')
    print('3')
def B():
    print('x')
    print('y')
    print('z')

generator 

In [12]:
def consumer():
    r=''
    while True:
        n=yield r 
        if not n:
            return
        print('[CONSUMER] Consuming %s...' %n)
        r='200 OK'
def produce(c):
    c.send(None)
    n=0
    while n<5:
        n=n+1 
        print('[PRODUCER] Producing %s...' %n)
        r=c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

In [13]:
c=consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


# asyncio

In [1]:
import asyncio

@asyncio.coroutine
def hello():
    print("hello world!")
    r= yield from asyncio.sleep(1)
    print("hello again!")

In [2]:
loop = asyncio.get_event_loop()

In [3]:
loop.run_until_complete(hello())
loop.close()

hello world!
hello again!


In [1]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop2 = asyncio.get_event_loop()
tasks=[hello(),hello()]
loop2.run_until_complete(asyncio.wait(tasks))
loop2.close()

Hello world! (<_MainThread(MainThread, started 13960)>)
Hello world! (<_MainThread(MainThread, started 13960)>)
Hello again! (<_MainThread(MainThread, started 13960)>)
Hello again! (<_MainThread(MainThread, started 13960)>)


In [1]:
import asyncio
@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect=asyncio.open_connection(host,80)
    reader,writer=yield from connect
    header='GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host,line.decode('utf-8').rstrip()))
    writer.close()

In [2]:
loop=asyncio.get_event_loop()
tasks=[wget(host) for host in ['www.sina.com.cn','www.sohu.com','www.163.com']]

In [3]:
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.sina.com.cn...
wget www.163.com...
wget www.sohu.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Sat, 23 Feb 2019 05:57:52 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 154
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=edge,s=cern.beijing.ha2ts4.211.nb.sinaedge.com,c=2001:da8:8001:833:54db:62d6:9115:4ed2;
www.sina.com.cn header > X-Via-Edge: 155090147294700541591410011021eabd476
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Sat, 23 Feb 2019 05:57:29 GMT
www.sohu.com header > Cache-Control: max-age=60
www.sohu.com header > X-From-Sohu: X-SRC-Cached
www.sohu.com header > Content-Encoding: gzip
www.sohu

# async/await

@asyncio.coroutine ->> async;

yield from ->> await

In [1]:
import asyncio
async def wget(host):
    print('wget %s...' % host)
    connect=asyncio.open_connection(host,80)
    reader,writer=await connect
    header='GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    await writer.drain()
    while True:
        line = await reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host,line.decode('utf-8').rstrip()))
    writer.close()

loop=asyncio.get_event_loop()
tasks=[wget(host) for host in ['www.sina.com.cn','www.sohu.com','www.163.com']]

loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.sina.com.cn...
wget www.sohu.com...
wget www.163.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Sat, 23 Feb 2019 06:04:08 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 154
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=edge,s=cern.beijing.ha2ts4.206.nb.sinaedge.com,c=2001:da8:8001:833:54db:62d6:9115:4ed2;
www.sina.com.cn header > X-Via-Edge: 1550901848898005415914100060277f539b2
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Sat, 23 Feb 2019 06:03:35 GMT
www.sohu.com header > Cache-Control: max-age=60
www.sohu.com header > X-From-Sohu: X-SRC-Cached
www.sohu.com header > Content-Encoding: gzip
www.sohu